## Conversion to ONNX from keras model using tf2onnx python api 

In [5]:
# get image
!wget -q https://github.com/onnx/tensorflow-onnx/blob/main/tests/ade20k.jpg


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import onnxruntime

img_path = 'ade20k.jpg'

img = image.load_img(img_path, target_size=(224, 224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

### Run the keras model

In [3]:
model = ResNet50(weights='imagenet')

preds = model.predict(x)
print('Keras Predicted:', decode_predictions(preds, top=3)[0])
model.save(os.path.join("./tmp", model.name))

Keras Predicted: [('n04285008', 'sports_car', 0.34477815), ('n02974003', 'car_wheel', 0.28764302), ('n03100240', 'convertible', 0.10070907)]
INFO:tensorflow:Assets written to: ./tmp\resnet50\assets


c:\Users\Albert\anaconda3\envs\ml_env\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


### Convert to ONNX using the Python API

In [4]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = model.name + ".onnx"

model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=output_path)
output_names = [n.name for n in model_proto.graph.output]

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


### Run the ONNX model

In [5]:
providers = ['CPUExecutionProvider']
m = rt.InferenceSession(output_path, providers=providers)
onnx_pred = m.run(output_names, {"input": x})

print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

ONNX Predicted: [('n04285008', 'sports_car', 0.34477764), ('n02974003', 'car_wheel', 0.2876437), ('n03100240', 'convertible', 0.100708835)]


### Convert to ONNX using the command line

In [7]:
!python -m tf2onnx.convert --opset 13 \
    --saved-model {os.path.join("./tmp", model.name)} \
    --output  {os.path.join("./tmp", model.name + ".onnx")}

c:\Users\Albert\anaconda3\envs\ml_env\lib\runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-05-26 15:48:21,776 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2022-05-26 15:48:27,406 - INFO - Signatures found in model: [serving_default].
2022-05-26 15:48:27,406 - WARNING - '--signature_def' not specified, using first signature: serving_default
2022-05-26 15:48:27,409 - INFO - Output names: ['predictions']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2022-05-26 15:48:29,901 - WARNING - From c:\Users\Albert\anaconda3\envs\ml_env\lib\site-packages\tf2onnx\tf_loader.py:711: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph